In [58]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

# Initialize SIFT detector
sift = cv2.SIFT_create()
# Capture video from webcam
cap = cv2.VideoCapture('monitoring.avi')

# Train image
trainImage = cv2.imread('monitoring.jpg', 0)
while True:
    # Read frame
    ret, frame = cap.read()

    if ret:
        # Convert frame to grayscale
        grayframe = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Extract keypoints and descriptors from frame and trainImage
        kp1, des1 = sift.detectAndCompute(grayframe, None)
        kp2, des2 = sift.detectAndCompute(trainImage, None)

        # FLANN matcher parameters
        FLANN_INDEX_KDTREE = 0
        index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)

        flann = cv2.FlannBasedMatcher(index_params, search_params)
        matches = flann.knnMatch(des1, des2, k=2)

        # Filter matches using Lowe's ratio test
        good = []
        for m, n in matches:
            if m.distance < 0.7 * n.distance:
                good.append(m)
        MIN_MATCH_COUNT = 10
        # Find homography matrix if enough matches are found
        if len(good) > 10:
            src_pts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
            dst_pts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
            matchesMask = mask.ravel().tolist()

            # Transform trainImage and draw keypoints and matches
            h, w = trainImage.shape
            pts = np.float32([ [0,0],[0,h-1],[w-1,h-1],[w-1,0] ]).reshape(-1,1,2)
            dst = cv2.perspectiveTransform(pts,M)

            draw_params = dict(matchColor=(255, 212, 229), singlePointColor=None, matchesMask=matchesMask, flags=2)
            img3 = cv2.drawMatches(frame, kp1, trainImage, kp2, good, None, **draw_params)

            # Display results
            cv2.imshow('Frame', img3)
            cv2.imshow('TrainImage', trainImage)


            k = cv2.waitKey(1) & 0xff
            if k == 27:
                break
        else:
            print("Not enough matches are found - %d/%d" % (len(good), MIN_MATCH_COUNT))

    else:
        break

# Close resources
cap.release()
cv2.destroyAllWindows()
